In [107]:

import polars as pl
from spatial_filtering import arrays, constants, direction_of_arrival
import importlib

importlib.reload(arrays)
importlib.reload(constants)
importlib.reload(direction_of_arrival)

meerkat_pos = pl.read_excel('meerkat positions.xlsx')
meerkat_pos = meerkat_pos.with_columns(distance_from_center_m=(pl.col('East')**2 + pl.col('North')**2 + pl.col('Up')**2).sqrt())
meerkat_pos_core = meerkat_pos.filter(pl.col('distance_from_center_m') < 500)
core_indices = [int(r[-2:]) for r in meerkat_pos_core['Antenna'].to_list()]
data_np = meerkat_pos_core[['East', 'North', 'Up']].to_numpy()

from spatial_filtering.arrays import Array
import numpy as np

array = arrays.Array(data_np)
N = array.num_antennas
f = 1.227574219e9              # Frequency 1.2276 GHz GPS L2 band.
wavelength = constants.c / f
snapshots = 2000


In [92]:
core_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 34,
 35,
 36]

In [112]:
from dadanalyse import read_dada
GPS_CHANNEL = 50
POL = 0
# J1644-4559_2024-02-16T11:21:22.092_29_73532270706688.dada
dada = read_dada("../gps.dada")
print(dada.get_header())
dada = dada.combined_data()



{'HEADER': 'DADA', 'HDR_VERSION': '1.0', 'HDR_SIZE': 4096, 'DADA_VERSION': '1.0', 'OBS_ID': 'None', 'FILE_SIZE': '3048214528', 'FILE_NUMBER': '0', 'UTC_START': '1708082482.092702505', 'MJD_START': '60356.47317236924195', 'OBS_OFFSET': '0', 'OBS_OVERLAP': '0', 'SOURCE': 'J1644-4559_Offset1', 'RA': '16:44:26.25', 'DEC': '-45:59:09.6', 'TELESCOPE': 'MeerKAT', 'INSTRUMENT': 'CBF-Feng', 'RECEIVER': 'L-band', 'FREQ': '1284000000.000000', 'BW': '856000000.000000', 'TSAMP': '4.7850467290', 'BYTES_PER_SECOND': '3424000000.000000', 'NBIT': 8, 'NDIM': 2, 'NPOL': 2, 'NCHAN': 64, 'NANT': 57, 'ORDER': 'TAFTP', 'INNER_T': 256, 'SYNC_TIME': '1708039531.000000', 'SAMPLE_CLOCK': '1712000000.000000', 'SAMPLE_CLOCK_START': '73532270706688', 'CHAN0_IDX': 1728, 'Lowest_freq (MHz)': 1217.125}


In [113]:
array.steering_vector([0,0], wavelength)

array([ 0.29380879-0.95586421j, -0.38789908+0.92170185j,
       -0.80562962-0.59241955j,  0.89094245+0.45411623j,
        0.63052321+0.77617039j, -0.54047739-0.84135854j,
       -0.7179779 +0.69606589j,  0.99995061-0.00993887j,
       -0.99152701+0.12990066j, -0.91580951+0.40161293j,
        0.22433012+0.97451321j, -0.95112676+0.30880072j,
        0.38387379-0.92338557j,  0.70552644+0.7086836j ,
        0.77828376-0.62791273j, -0.49022161+0.87159783j,
        0.9929999 -0.1181152j ,  0.07410256-0.99725063j,
       -0.93833154-0.34573679j,  0.08121442-0.99669665j,
        0.67891664-0.73421536j,  0.94433605+0.3289824j ,
        0.53076907+0.84751649j,  0.01367786+0.99990645j,
        0.76778561+0.64070684j, -0.99951007+0.03129872j,
       -0.10965214-0.99397002j, -0.98281239+0.18460718j,
        0.31836857+0.94796701j, -0.86498433+0.50179887j,
        0.5036235 -0.86392324j, -0.07489689+0.99719128j,
        0.66157129-0.74988227j, -0.14029655-0.99010953j])

In [114]:
dada.shape

(208896, 57, 64, 2)

In [115]:
dada = dada[0:2000, core_indices, GPS_CHANNEL, POL].T
X = dada

In [97]:
from skyfield.api import load, Topos, EarthSatellite
from datetime import datetime
import numpy as np

# Location: MeerKAT Telescope (converted to decimal degrees)
lat = -30.71106
lon = 21.44389
elevation_m = 1086.6

# Time: Convert UNIX timestamp to UTC
timestamp = 1708082482.092702505
dt = datetime.utcfromtimestamp(timestamp)
dt

lon_str = "21d26m38s"   # degrees, minutes, seconds format
lat_str = "-30d42m39.8s"  # negative for south

# Altitude (optional, in meters)
height = 1086.6 * u.m

# Create location
location = EarthLocation.from_geodetic(lon=lon_str, lat=lat_str, height=height)

print(location)

(5109360.133321234, 2006852.5860429062, -3238948.1274788757) m


In [98]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u

# Observer location (MeerKAT)
obs_location = EarthLocation(lat=lat*u.deg,
                         lon=lon*u.deg,
                         height=elevation_m*u.m)

# Time of observation (UTC)
obstime = Time('2024-02-16 11:21:22')

# RA/DEC coordinates
ra_str = '16:44:26.25'
dec_str = '-45:59:09.6'

# Create SkyCoord object in ICRS (equatorial)
sky_coord = SkyCoord(ra=ra_str, dec=dec_str, unit=(u.hourangle, u.deg), frame='icrs')

# Create AltAz frame at observer location and time
altaz_frame = AltAz(obstime=obstime, location=location)

# Transform RA/DEC to AltAz
altaz = sky_coord.transform_to(altaz_frame)

# Output altitude and azimuth
print("Source RA/DEC")
print(f"Altitude: {altaz.alt:.2f}")
print(f"Azimuth: {altaz.az:.2f}")

theta_source = 90 - altaz.alt.degree
phi_source = (90 - altaz.az.degree) % 360

Source RA/DEC
Altitude: 24.03 deg
Azimuth: 229.33 deg


In [99]:
array_factor = array.steering_vector([np.deg2rad(phi_source), np.deg2rad(theta_source)], wavelength)
#X = X.T
X_adj = X.copy()
for i, shift in enumerate(array_factor):
    X_adj[i, :] = X_adj[i, :] * array_factor[i].conj()

R = X @ X.conj().T / (N-1)
R_adj = X_adj @ X_adj.conj().T / (N-1)

In [100]:
import numpy as np

evals, evecs = np.linalg.eigh(R)
R

array([[57253.85     +0.j    , -1857.6365-7104.485j ,
         4543.606 +4202.8486j, ...,  6561.7275+1706.0304j,
        -5614.4546-6793.8486j, -3782.5457-6208.576j ],
       [-1857.6365+7104.485j , 53519.457    +0.j    ,
        -4328.091 +2280.6062j, ..., -3793.7576+5632.9697j,
         6008.6064-6115.6367j,  1823.1819+7210.0913j],
       [ 4543.606 -4202.8486j, -4328.091 -2280.6062j,
        59934.426    +0.j    , ..., 12704.909 -4904.0303j,
        -6734.576 +8737.939j , -7210.485 -6355.788j ],
       ...,
       [ 6561.7275-1706.0304j, -3793.7576-5632.9697j,
        12704.909 +4904.0303j, ..., 57076.7      +0.j    ,
        -8230.151 +5723.182j , -4744.0303-9713.j    ],
       [-5614.4546+6793.8486j,  6008.6064+6115.6367j,
        -6734.576 -8737.939j , ..., -8230.151 -5723.182j ,
        59026.73     +0.j    , -6225.9395+4561.6367j],
       [-3782.5457+6208.576j ,  1823.1819-7210.0913j,
        -7210.485 +6355.788j , ..., -4744.0303+9713.j    ,
        -6225.9395-4561.6367j, 5873

In [101]:
R_adj

array([[57253.85   +0.0000000e+00j,  4803.674  +5.5542090e+03j,
         5374.2246 +3.0701787e+03j, ...,  5310.684  +4.2146694e+03j,
         8811.456  -1.9161501e+02j, -3871.8713 +6.1532642e+03j],
       [ 4803.674  -5.5542090e+03j, 53519.457  +0.0000000e+00j,
         4733.909  -1.2343584e+03j, ...,  6754.485  -7.0701495e+02j,
         2794.2422 -8.1053423e+03j, -7201.537  +1.8566796e+03j],
       [ 5374.2246 -3.0701787e+03j,  4733.909  +1.2343584e+03j,
        59934.426  +0.0000000e+00j, ..., 13104.872  +3.7049373e+03j,
         -196.84476-1.1030293e+04j, -4506.0923 +8.4901279e+03j],
       ...,
       [ 5310.684  -4.2146694e+03j,  6754.485  +7.0701495e+02j,
        13104.872  -3.7049373e+03j, ..., 57076.7    +0.0000000e+00j,
        -3486.3782 -9.3986885e+03j, -2548.6433 +1.0504885e+04j],
       [ 8811.456  +1.9161501e+02j,  2794.2422 +8.1053423e+03j,
         -196.84476+1.1030293e+04j, ..., -3486.3782 +9.3986885e+03j,
        59026.73   +0.0000000e+00j, -1475.7306 -7.5758213e+03j]

In [102]:
import matplotlib.pyplot as plt

plt.plot(evals)
plt.show()

can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [103]:
evals[-15:]

array([ 44724.496,  45110.02 ,  46194.11 ,  47069.992,  47335.273,
        48223.836,  49212.336,  50403.38 ,  51309.844,  54316.02 ,
        58430.457,  69177.12 , 120083.95 , 159751.81 , 329082.1  ],
      dtype=float32)

In [104]:
360 / 12000

0.03

In [108]:


theta_steps = 6_000
phi_steps = 24_000

output = direction_of_arrival.MUSICDOA2D().get_directions(
    array,
    R_adj, 
    num_interferers=6,
    wavelength = wavelength,
    theta_min_deg = 0,
    theta_max_deg = 86,
    theta_steps=theta_steps,
    phi_min_deg = 0,
    phi_max_deg = 360,
    phi_steps=phi_steps,
)

100%|███████████████████████████████████████████████████| 144000000/144000000 [4:00:46<00:00, 9968.10it/s]


In [110]:
output.write_parquet('output.parquet')

In [36]:

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'  

# Filter top 1% Q values (adjust as needed)
threshold = output['Q'].quantile(0.999975)
high_Q = output.filter(pl.col('Q') >= threshold)

# Create 3D scatter plot
fig = px.scatter_3d(
    high_Q,
    x='theta',  # azimuth
    y='phi',      # range
    z='Q',      # MUSIC power
    color='Q',
    color_continuous_scale='Jet',
    opacity=0.8
)

# Update layout for better visuals
fig.update_layout(
    title='High-Q MUSIC Spectrum Peaks',
    scene=dict(
        xaxis_title='Angle (degrees)',
        yaxis_title='Phi (degrees)',
        zaxis_title='Q (dB)'
    )
)

fig.show()

The MJD I want is 60356.47317236924195




In [63]:
import util
importlib.reload(util)

tles_check = []
with open('tles_sgp4.tle', 'r') as f:
    while True:
        line1 = f.readline()
        if not line1 or line1.strip() == "":
            break  # stop if first line is empty

        line2 = f.readline()
        if not line2 or line2.strip() == "":
            break  # stop if second line is empty
        
        tles_check.append(util.TLE(line1.strip(), line2.strip()))

sat_check = []
for tle in tles_check:
    sat_check.append(util.get_position_at_time(tle, obstime, location))

sat_check

[(<Latitude 49.9011741 deg>, <Longitude 140.06565009 deg>),
 (<Latitude 16.64217106 deg>, <Longitude 91.63357231 deg>),
 (<Latitude -14.3998069 deg>, <Longitude 89.75286887 deg>),
 (<Latitude -15.55078374 deg>, <Longitude 226.63380823 deg>),
 (<Latitude -40.42825261 deg>, <Longitude 48.57907833 deg>),
 (<Latitude -76.46595843 deg>, <Longitude 213.61790564 deg>),
 (<Latitude -51.08696114 deg>, <Longitude 72.08651475 deg>),
 (<Latitude -53.13859519 deg>, <Longitude 285.93121387 deg>),
 (<Latitude -9.69607389 deg>, <Longitude 2.15128416 deg>),
 (<Latitude 76.9560202 deg>, <Longitude 197.63728034 deg>),
 (<Latitude 37.98097699 deg>, <Longitude 309.60795381 deg>),
 (<Latitude -17.82041055 deg>, <Longitude 159.98671405 deg>),
 (<Latitude -11.11461928 deg>, <Longitude 333.78781491 deg>),
 (<Latitude 36.85778348 deg>, <Longitude 20.40300613 deg>),
 (<Latitude -10.52907699 deg>, <Longitude 203.35030503 deg>),
 (<Latitude 4.09678194 deg>, <Longitude 141.1021063 deg>),
 (<Latitude -21.36389631 de

In [64]:
df

name,alt,az
str,f64,f64
"""NAVSTAR 47 (USA 150)""",16.642376,91.633704
"""NAVSTAR 81 (USA 319)""",8.048966,45.01476
"""NAVSTAR 43 (USA 132)""",49.903986,140.065598
"""NAVSTAR 60 (USA 196)""",76.956355,197.642354
"""NAVSTAR 61 (USA 199)""",37.980936,309.60797
"""NAVSTAR 67 (USA 239)""",36.857815,20.403034
"""NAVSTAR 78 (USA 293)""",45.372665,219.380969
"""NAVSTAR 79 (USA 304)""",22.556092,245.454394
"""NAVSTAR 69 (USA 248)""",4.096493,141.102433


In [38]:
from skyfield.api import load, EarthSatellite
from skyfield.iokit import parse_tle_file
import json
ts = load.timescale()

with load.open('gp2_2.tle') as f:
    data = json.load(f)

ts = load.timescale()
sats = [EarthSatellite.from_omm(ts, fields) for fields in data]

print('Loaded', len(data), 'satellites')

Loaded 54 satellites


In [41]:
from skyfield.api import wgs84, Topos

meerkat = Topos(latitude_degrees=-30.71106,
                longitude_degrees=21.44389,
                elevation_m= 1086.6)

obs_site = meerkat
t_obs = ts.utc(2024, 2, 16, 11, 21, 22)
sats_visible = []
sats_processed = set()

for satellite in sats:
    # There are some duplicate TLEs - don't worry about this.
    # any of them should give essentially the same result.
    if satellite.name in sats_processed:
        continue
    event_dict = {}
    event_dict['name'] = satellite.name

    topocentric = (satellite - obs_site).at(t_obs)
    alt, az, _ = topocentric.altaz()
    event_dict['alt'] = alt.degrees
    event_dict['az'] = az.degrees
    if alt.degrees > 0:
        sats_visible.append(event_dict)
    sats_processed.add(satellite.name)

df = pl.from_records(sats_visible)
#df.write_excel('sats.xlsx')
#len(sats_processed)
print(len(sats_visible))
df.with_columns(theta=pl.lit(90) - pl.col('alt'), phi=(pl.lit(90) - pl.col('az'))%360).sort(by="alt", descending=False)


10


name,alt,az,theta,phi
str,f64,f64,f64,f64
"""NAVSTAR 69 (USA 248)""",4.096493,141.102433,85.903507,308.897567
"""NAVSTAR 81 (USA 319)""",8.048966,45.01476,81.951034,44.98524
"""NAVSTAR 47 (USA 150)""",16.642376,91.633704,73.357624,358.366296
"""NAVSTAR 79 (USA 304)""",22.556092,245.454394,67.443908,204.545606
"""NAVSTAR 67 (USA 239)""",36.857815,20.403034,53.142185,69.596966
"""NAVSTAR 61 (USA 199)""",37.980936,309.60797,52.019064,140.39203
"""NAVSTAR 64 (USA 206)""",44.369909,123.403165,45.630091,326.596835
"""NAVSTAR 78 (USA 293)""",45.372665,219.380969,44.627335,230.619031
"""NAVSTAR 43 (USA 132)""",49.903986,140.065598,40.096014,309.934402


In [111]:
output.with_columns(dist_from_theta=90 - pl.col('theta'), dist_from_phi = (90-pl.col('phi')) % 360).sort(by="Q", descending=True).head(10)

phi,theta,Q,dist_from_theta,dist_from_phi
f64,f64,f64,f64,f64
171.59215,82.043341,0.0,7.956659,278.40785
147.12613,56.726454,-0.081025,33.273546,302.87387
171.59215,82.057676,-0.082729,7.942324,278.40785
110.959623,48.024671,-0.404396,41.975329,339.040377
171.59215,82.029005,-0.407285,7.970995,278.40785
22.395933,75.87898,-0.491394,14.12102,67.604067
22.395933,75.893316,-0.559598,14.106684,67.604067
359.834993,74.646108,-0.567958,15.353892,90.165007
60.932539,62.360393,-0.577288,27.639607,29.067461


In [163]:
from astropy.time import Time

# Your MJD value (example)
mjd = 60356.47317236924195

# Create Time object
t = Time(mjd, format='mjd', scale='utc')

# Print UTC as ISO format
print("UTC Time:", t.utc.iso)

# Or get components
print("Year:", t.utc.datetime.year)
print("Datetime:", t.utc.datetime)

UTC Time: 2024-02-16 11:21:22.093
Year: 2024
Datetime: 2024-02-16 11:21:22.092702


In [164]:
## Near-field limit calculation
1500**2 / (constants.c / 1.276e9)

9576625.173138944